In [1]:
import matplotlib.pyplot as plt
import aplpy
from astropy.wcs import WCS
from astropy.io import fits, ascii
from astropy.utils.data import get_pkg_data_filename
from astroquery.skyview import SkyView
import ssl
import aplpy
from lens import Lens
from astropy.table import Table
import sqlutil as sqlutil
import numpy as np
import lensutils as util


candidates = Table.read('data/candidatesDistTime.txt',format='ascii')
#candidates = Table.read('data/largpmcandidates.txt',format='ascii')
timeMask = (candidates['Closest_approah_time[JYr]'] > 2018.0) & (candidates['Closest_approach_Dist[mas]'] < 260.0)

candidates = candidates[timeMask]
TGASlensid = candidates['TGAS_id']
ppmxlid = candidates['ppmxl_id']
#CloseApp = candidates['Closest_approach_Dist[mas]'] 
#timeApp = candidates['Closest_approah_time[JYr]']
#imsize = 2

for i in range(0,len(TGASlensid)):
    
    querystringTGAS = 'select ra, dec, source_id,pmra,pmdec,ref_epoch, phot_g_mean_mag from gaia_dr1.tgas_source where source_id=' + str(TGASlensid[i])
    querystringPPMXL ='select ra, dec, id, epochra, epochdec,jmag,hmag,kmag,b1mag,b2mag,r1mag,r2mag from ppmxl.main where id=' + str(ppmxlid[i])

    lensRa, lensDec, id,pmra,pmdec,ref_epoch, gmag = sqlutil.get(querystringTGAS,
                                                           db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')

    testlens1 = Lens(id[0],lensRa[0],lensDec[0],pmra[0],pmdec[0],ref_epoch[0])

    sourceRa, sourceDec,sourceId, epochra, epochdec, jmag,hmag,kmag,b1mag,b2mag,r1mag,r2mag = sqlutil.get(querystringPPMXL,
                       db='wsdb',host='cappc127.ast.cam.ac.uk', user='peter_mcgill', password='Ln3g.wsk')

    util.plt_lens_env(testlens1,sourceRa[0],sourceDec[0],sourceId[0],[jmag[0],hmag[0],kmag[0],b1mag[0],b2mag[0],r1mag[0],r2mag[0]],gmag[0])
    
    
    


INFO: Auto-setting vmin to  2.266e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.316e+04 [aplpy.core]
INFO: Auto-setting vmin to  1.988e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.438e+04 [aplpy.core]
INFO: Auto-setting vmin to  1.392e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.703e+04 [aplpy.core]
INFO: Auto-setting vmin to  6.393e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.983e+04 [aplpy.core]
INFO: Auto-setting vmin to  1.601e+03 [aplpy.core]
INFO: Auto-setting vmax to  2.584e+04 [aplpy.core]
